__[pydantic fields](https://github.com/apowers313/roc/blob/master/experiments/2025.01.06-16.50.23-pydantic-fields/2025.01.06-16.50.23-pydantic-fields.ipynb)__

In [ ]:
!date

In [ ]:
# save notebook path before we get started
import os

notebook_path = os.path.abspath("")  # not sure if this or os.getcwd() is more reliable

# Enumerating Fields

In [2]:
from pydantic import BaseModel


class ClsOne(BaseModel):
    name: str


class ClsTwo(ClsOne):
    age: int


print("ClsOne fields", ClsOne.model_fields)
print("ClsTwo fields", ClsTwo.model_fields)

ClsOne fields {'name': FieldInfo(annotation=str, required=True)}
ClsTwo fields {'name': FieldInfo(annotation=str, required=True), 'age': FieldInfo(annotation=int, required=True)}


# Setting Field Default

In [10]:
from pydantic import BaseModel, Field


class ClsOne(BaseModel):
    name: str


class ClsTwo(ClsOne):
    name: str = Field(default="foo")


two = ClsTwo()
print("two:", two)
print("two name:", two.name)

two: name='foo'
two name: foo


# Getting Field Default

In [35]:
from pydantic import BaseModel, Field
from datetime import datetime


class ClsOne(BaseModel):
    name: str = "foo"
    timestamp: datetime = Field(default_factory=datetime.now, frozen=True)


class ClsTwo(ClsOne):
    age: int = Field(default=42)


class ClsThree(ClsTwo):
    # timestamp: datetime = datetime.now()
    pass


print("ClsOne fields", ClsOne.model_fields)
print("ClsTwo fields", ClsTwo.model_fields)
print("ClsTwo fields age", ClsTwo.model_fields["age"])
print("ClsTwo fields age class type", ClsTwo.model_fields["age"].__class__.__name__)
print("ClsTwo fields age default", ClsTwo.model_fields["age"].default)
print("ClsTwo fields age default", ClsTwo.model_fields["age"].get_default())
print(
    "ClsTwo fields datetime default",
    ClsTwo.model_fields["timestamp"].get_default(call_default_factory=True),
)
three = ClsThree()
three.timestamp = datetime.fromtimestamp(0)
print("three", three)
one = ClsOne()
one.timestamp = datetime.fromtimestamp(100000)
print("one", one)

ClsOne fields {'name': FieldInfo(annotation=str, required=False, default='foo'), 'timestamp': FieldInfo(annotation=datetime, required=False, default_factory=now, frozen=True)}
ClsTwo fields {'name': FieldInfo(annotation=str, required=False, default='foo'), 'timestamp': FieldInfo(annotation=datetime, required=False, default_factory=now, frozen=True), 'age': FieldInfo(annotation=int, required=False, default=42)}
ClsTwo fields age annotation=int required=False default=42
ClsTwo fields age class type FieldInfo
ClsTwo fields age default 42
ClsTwo fields age default 42
ClsTwo fields datetime default 2025-01-06 21:25:59.761942


ValidationError: 1 validation error for ClsThree
timestamp
  Field is frozen [type=frozen_field, input_value=datetime.datetime(1969, 12, 31, 16, 0), input_type=datetime]
    For further information visit https://errors.pydantic.dev/2.10/v/frozen_field

In [6]:
from __future__ import annotations

from pydantic import BaseModel, Field


class Node(BaseModel):
    labels: set[str] | None = None
    name: str = "Bob"

    def __init__(self) -> None:
        super().__init__()
        print("Node init:", self.__class__.__name__)

    def __init_subclass__(cls, **kwargs: Any) -> None:
        super().__init_subclass__(**kwargs)
        print("Node Subclass init:", cls.__name__)
        # cls.name = Field(default="Sam")
        # print("type hints", get_type_hints(cls, include_extras=True))
        # print("type hints", cls.__annotations__)
        # print("cls dict", cls.__dict__)
        print("cls labels", hasattr(cls, "labels"))
        print("cls model_fields", cls.model_fields)


class Foo(Node):
    def __init__(self) -> None:
        super().__init__()
        print("Foo init")


class Bar(Foo):
    def __init__(self) -> Node:
        super().__init__()
        print("Bar init")


b = Bar()
print("bob", b.name)
print("Node subclasses", Node.__subclasses__())
print("Foo subclasses", Foo.__subclasses__())
print("Bar bases", Bar.__bases__)
print("Bar base", Bar.__base__)
print("Bar mro", Bar.__mro__)

Node Subclass init: Foo
cls labels False
cls model_fields {'labels': FieldInfo(annotation=Union[set[str], NoneType], required=False, default=None), 'name': FieldInfo(annotation=str, required=False, default='Bob')}
Node Subclass init: Bar
cls labels False
cls model_fields {'labels': FieldInfo(annotation=Union[set[str], NoneType], required=False, default=None), 'name': FieldInfo(annotation=str, required=False, default='Bob')}
Node init: Bar
Foo init
Bar init
bob Bob
Node subclasses [<class '__main__.Foo'>]
Foo subclasses [<class '__main__.Bar'>]
Bar bases (<class '__main__.Foo'>,)
Bar base <class '__main__.Foo'>
Bar mro (<class '__main__.Bar'>, <class '__main__.Foo'>, <class '__main__.Node'>, <class 'pydantic.main.BaseModel'>, <class 'object'>)


In [5]:
import os

from pydantic import BaseModel, ValidationError

from pydantic_settings import BaseSettings, SettingsConfigDict

class Config(BaseSettings):
    model_config = SettingsConfigDict(env_prefix='my_prefix_', env_file=".env", extra="forbid")
    foo: list[str]

try:
    os.remove(".env")
except Exception:
    pass

# first: try parsing a list from an environment variable with a prefix
list_value = '["testy"]'
os.environ['my_prefix_foo'] = '["testy"]'
print("First Config", Config())
print("is list", isinstance(Config().foo, list))
# output:
# First Config foo=['testy']
# is list True
del os.environ['my_prefix_foo']
print("\n") 

# second: try parsing a list from an environment variable with a prefix
list_value = '["testy"]'
os.environ['foo'] = '["testy"]'
try:
    Config()
except ValidationError as e:
    print("Second Config:", e)
# output:
# Second Config: 1 validation error for Config
# foo
#   Input should be a valid list [type=list_type, input_value='["testy"]', input_type=str]
#     For further information visit https://errors.pydantic.dev/2.10/v/list_type
del os.environ['foo'] 
print("\n") 

# third: try parsing a list from a .env variable with a prefix
with open(".env", "w") as file:
    file.write(f"my_prefix_foo={list_value}")
print("Third Config", Config())
print("is list", isinstance(Config().foo, list))
# output
# Third Config foo=['testy']
# is list True
os.remove(".env")
print("\n") 

# fourth: try parsing a list from a .env variable WITHOUT a prefix
with open(".env", "w") as file:
    file.write(f"foo={list_value}")
try:
    Config()
except ValidationError as e:
    print("Fourth Config:", e)
# Fourth Config: 1 validation error for Config
# foo
#   Input should be a valid list [type=list_type, input_value='["testy"]', input_type=str]
#     For further information visit https://errors.pydantic.dev/2.10/v/list_type
os.remove(".env")


First Config foo=['testy']
is list True


Second Config: 1 validation error for Config
foo
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing


Third Config foo=['testy']
is list True


Fourth Config: 1 validation error for Config
foo
  Input should be a valid list [type=list_type, input_value='["testy"]', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/list_type
